In [2]:
from loader import list_files, Files, Folders  #precalculate what we are loading in
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sys import getsizeof
from data_xml import Loader

In [4]:

class dataset:
    def __init__(self,files,temporal_window,delay=0,compression_dim=1,crop=None):
        self.X=[]
        self.y=[]
        self.y2=[]
        new_dim=(int(640*compression_dim),int(480*compression_dim))
        if type(crop)!=type(None):
            crop_x=abs(crop[0]-crop[1])
            crop_y=abs(crop[2]-crop[3])
        for i in range(len(files)): #loop through all files 
            loader_=Loader(Files[i])
            frame=loader_.getByExperiment() #get frame
            del loader_
            for index, row in frame.iterrows():
                folder=files[i].replace(".xml","_"+row['Readings']+".npz")
                data = np.load(folder) #load data
                for array_name in data:
                    data=data[array_name].reshape(100,480,640,3)
                    window=data[::4][delay:delay+temporal_window]
                    a=np.zeros((len(window),new_dim[1],new_dim[0],))
                    if type(crop)!=type(None):
                        a=np.zeros((len(window),crop_y,crop_x,))
                    if compression_dim<1:
                        for j,frame in enumerate(window):
                            image=cv2.resize(frame,new_dim,interpolation=cv2.INTER_AREA) #resize to new dimentions
                            if type(crop)!=type(None): a[j]=cv2.cvtColor(image[crop[2]:crop[3],crop[0]:crop[1]],cv2.COLOR_BGR2GRAY) #crop out centre
                            else: a[j]=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                            #a[j] = cv2.cvtColor(a[j], cv2.COLOR_BGR2GRAY)
                    window=a.copy()
                    self.y.append(row['Positions'])
                    self.y2.append(files[i])
                    self.X.append(window)
                clear()
                print("Dataset size:",len(self.X),"\nWindow size:",self.X[0].shape[0],"\nImage:",self.X[0].shape[1:])
                print("Memory needed:",round(self.getSize(self.X)/ 1024 / 1024/ 1024,2),"GB")
                self.append_to_npz("/its/home/drs25/Documents/data/Tactile Dataset/directions_X_data",np.array(self.X))
                self.np.savez_compressed("/its/home/drs25/Documents/data/Tactile Dataset/directions_y_data",np.array(self.y))
                self.X=[]
                self.y=[]
                self.y2=[]
        self.X=np.array(self.X)
        self.y=np.array(self.y)
        #self.y2=np.array(self.y2)
        print("Dataset size:",self.X.shape[0],"\nWindow size:",self.X.shape[1],"\nImage:",self.X.shape[2:])
        print("Memory needed:",round(getsizeof(self.X)/ 1024 / 1024/ 1024,2),"GB")
    def append_to_npz(self,npz_file, **new_data):
        try:
            # Load existing data
            with np.load(npz_file, allow_pickle=True) as data:
                existing_data = {key: data[key] for key in data}

            # Concatenate new data to the existing arrays
            for key, array in new_data.items():
                if key in existing_data:
                    existing_data[key] = np.concatenate((existing_data[key], array))
                else:
                    existing_data[key] = array

            # Save updated data to a new .npz file
            np.savez_compressed(npz_file, **existing_data)

        except FileNotFoundError:
            # If file does not exist, just save the new data
            np.savez_compressed(npz_file, **new_data)
    def getSize(self,npz_file):
        try:
            file_size = os.path.getsize(npz_file)
            #print(f"Size of '{npz_file}': {file_size} bytes")
            return file_size
        except FileNotFoundError:
            print(f"File '{npz_file}' not found.")
            return None

data=dataset(Files,20,compression_dim=0.4,delay=0,crop=[60,180,40,150]) #20 steps approximatly 4 seconds
plt.imshow(data.X[0][0].astype(int))
plt.title(data.keys[data.y[0]])
plt.show()
np.savez_compressed("/its/home/drs25/Documents/data/Tactile Dataset/directions_X_data",data.X)
np.savez_compressed("/its/home/drs25/Documents/data/Tactile Dataset/directions_y_data",data.y)